# Lesson 9 - Adding MCP to the Hospital Server

In this final lesson, you will add another agent to the Hospital Server that uses MCP to get access to tools:
- you will first define an MCP server using `FastMCP` that runs using the `stdio` transport (i.e., it runs locally) and exposes one tool;
- you will then update the hospital server to include a second agent;
- you will define the agent using the ToolCallingAgent of Smolagents and pass to it the tool exposed by the MCP server;
- you will finally activate the agent and interact with it using an ACP client.

## 9.1. Output MCP Server

Run this cell to create the file `mcpserver.py` inside the folder `my_acp_project`. The dependencies in the `my_acp_project` have been also updated so you can run this MCP server using `uv run`. Please check the note at the end of the lesson to learn more about these additional dependencies.

## 9.2. Updating the Hospital Server to use MCP

You will add the definition of the second agent `doctor_agent` which helps users find doctors near them. Note that for the MCP server, the command is defined as `uv run`.

In [1]:
!pip install pysqlite3-binary crewai crewai_tools acp-sdk load_dotenv uv nest-asyncio colorama smolagents --quiet
# pysqlite3 sqlite-vss
!pip install chromadb ollama langchain_huggingface sentence-transformers google-generativeai --quiet
!pip install mcpadapt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.53.2 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
vllm 0.9.2 requires tokenizers>=0.21.1, but you have tokenizers 0.20.3 which is incompatible.
appengine-python-standard 1.1.9 requires urllib3<2,>=1.26.2, but you have urllib3 2.5.0 which is incompatible.
codeflare-sdk 0.26.0 requires pydantic<2, but you have pydantic 2.11.7 which is incompatible.
codeflare-sdk 0.26.0 requires ray[data,default]==2.35.0, but you have ray 2.48.0 which is incompatible.
kfp 2.9.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.
kfp 2.9.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
kfp 2.9.0 requires urllib3<2.0.0, but you have urllib3 2.5.0 which is incompatible.
kfp-kubernetes 1.4.0 requires p

In [2]:
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"

In [ ]:
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import RunYield, RunYieldResume, Server
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool, ToolCallingAgent, ToolCollection
from mcp import StdioServerParameters

import nest_asyncio
nest_asyncio.apply()

server = Server()

from smolagents import OpenAIServerModel

# Configure the model to use LM Studio's local API endpoint
model = OpenAIServerModel(
    model_id= "ibm-granite/granite-3.3-2b-instruct",  # This can be any name, LM Studio will use whatever model you have loaded
    api_base= f"{INFERENCE_SERVER_URL}/v1",  # Default LM Studio API endpoint
    api_key=API_KEY # LM Studio doesn't require an API key by default
)


server_parameters = StdioServerParameters(
    command="uv",
    args=["run", "./mcp_project/research_server.py"],
    env=None,
)

@server.agent()
async def research_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is a Doctor Agent which helps users find doctors near them."
    with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
        agent = ToolCallingAgent(tools=[*tool_collection.tools], model=model)
        prompt = input[0].parts[0].content
        response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])

if __name__ == "__main__":
    server.run(port=8003)

INFO:     Started server process [2144]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8003 (Press CTRL+C to quit)
INFO:     Run started


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search redhat blogs?                                                                                            │
│                                                                                                                 │
╰─ OpenAIServerModel - ibm-granite/granite-3.3-2b-instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

INFO:     HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'search_papers' with arguments: {'topic': 'redhat blogs', 'max_results': 5}                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Observations: The paper titled 'Where and What do Software Architects blog? An Exploratory Study on Architectural 
Knowledge in Blogs, and their Relevance to Design Steps' provides an exploratory study on architectural blogs where
software engineers share their architectural knowledge (AK). The research examines the types, topics, and relevance
of these blogs to software design steps, and supports researchers and practitioners in finding and reusing AK from 
architectural blogs.

Final answer: The paper titled 'Where and What do Software Architects blog? An Exploratory Study on Architectural 
Knowledge in Blogs, and their Relevance to Design Steps' provides an exploratory study on architectural blogs where
software engineers share their architectural knowledge (AK). The research examines the types, topics, and relevance
of these blogs to software design steps, and supports researchers and practitioners in finding and reusing AK from 
architectural blogs.

[Step 3: Duration 1.83 seconds| Input tokens: 5,712 | Output tokens: 203]

INFO:     Run completed


INFO:     127.0.0.1:35284 - "POST /runs HTTP/1.1" 200 OK


## 9.3. Run the Hospital ACP x MCP Server

- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you can stop it by typing `CTRL+C`.
- Run the server again by typing:
  - `uv run smolagents_server.py`

## 9.4. Call the ACPxMCP Server

**Note: How to update the `my_acp_project` locally on your machine so you can enable MCP?**
- cd `my_acp_project`
- `uv add mcp smolagents[mcp]`

## 9.4. Resources

- [Smolagents from mcp tool](https://huggingface.co/docs/smolagents/en/reference/tools#smolagents.ToolCollection.from_mcp)
- [MCP short course](https://www.deeplearning.ai/short-courses/mcp-build-rich-context-ai-apps-with-anthropic/)
- [MCP and A2A](https://agentcommunicationprotocol.dev/about/mcp-and-a2a)
- [The doctors json file used accessed through the MCP server](https://raw.githubusercontent.com/nicknochnack/ACPWalkthrough/refs/heads/main/doctors.json)